In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.types as t

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Spark_movies_countries_task") \
    .config("spark.master", "spark://ZHUYANZHAO-HWPC.localdomain:7077") \
    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.22") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.num", "1") \
    .getOrCreate()

In [ ]:
df_movies = spark.read.option("header","true").csv("./movies_metadata.csv")
df_movie_prod_countries = df_movies.select(f.col("id").alias("movie_tmdb_id"), f.col("production_countries"))
df_movie_prod_countries = df_movie_prod_countries.withColumn("production_countries", f.from_json(f.col("production_countries"), "array<string>"))
df_movie_prod_countries = df_movie_prod_countries.na.drop(subset=['production_countries'])
df_movie_prod_countries = df_movie_prod_countries.withColumn("production_countries", f.explode("production_countries"))
df_movie_prod_countries.printSchema()
df_movie_prod_countries.show(10, truncate=False)

In [ ]:
schema = t.StructType(
    [
        t.StructField('iso_3166_1', t.StringType(), True),
        t.StructField('name', t.StringType(), True)
    ]
)
df_movie_prod_countries = df_movie_prod_countries.withColumn('production_countries', f.from_json('production_countries', schema))\
.select(f.col('movie_tmdb_id'), f.col('production_countries.iso_3166_1').alias('country_code'), f.col('production_countries.name'))
df_movie_prod_countries = df_movie_prod_countries.na.drop('any')
df_movie_prod_countries.printSchema()
df_movie_prod_countries.show(10, truncate=False)

In [ ]:
df_movie_prod_countries.select(*df_movie_prod_countries.columns).write.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/spark_movies") \
    .option("driver", "com.mysql.cj.jdbc.Driver").option("dbtable", "movie_prod_countries") \
    .option("user", "root").option("password", "zhuyz928").save()

In [ ]:
df_prod_countries = df_movie_prod_countries.select(['country_code', 'name']).distinct()
df_prod_countries.sort(f.col('country_code'))

In [ ]:
df_prod_countries.select(*df_prod_countries.columns).write.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/spark_movies") \
    .option("driver","com.mysql.cj.jdbc.Driver").option("dbtable", "prod_countries") \
    .option("user", "root").option("password", "zhuyz928").save()